In [ ]:
# Install / setup cell (RUN FIRST)
import sys
!{sys.executable} -m pip install -U \
  langchain \
  langchain-core \
  langchain-community \
  langchain-text-splitters \
  langchain-huggingface \
  langchain-google-vertexai

⬆️ **After this cell completes:**

**Restart the kernel NOW**, then continue below.

Yes, restarting twice is intentional when upgrading LangChain.

In [ ]:
# Core imports (LangChain 1.x compatible)
import time
import os
import warnings

# Suppress tqdm warning about ipywidgets
warnings.filterwarnings('ignore', message='IProgress not found')

from langchain_text_splitters import CharacterTextSplitter
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, SystemMessage

from langchain_google_vertexai import (
    ChatVertexAI,
    VertexAIEmbeddings,
    VertexAI,
)

print('Environment ready')

## Step 1: Define metadata wrapper function

This function formats and adds metadata to chunks for the Oracle Vector Store.

In [ ]:
# Function to format and add metadata to Oracle 23ai Vector Store
from langchain_core.documents import Document

def chunks_to_docs_wrapper(row: dict) -> Document:
    """
    Converts text into a Document object suitable for ingestion into Oracle Vector Store.
    """
    metadata = {'id': row['id'], 'link': row['link']}
    return Document(page_content=row['text'], metadata=metadata)

print("Successfully defined metadata wrapper")

## Step 2: Connect to Oracle Database 23ai

Update with your credentials:
- **Username**
- **Password** 
- **Connection String** (from tnsnames.ora)
- **Wallet Password**

In [ ]:
import oracledb

un = "ADMIN"           # Enter Username
pw = "password"           # Enter Password
dsn = 'connection string' # Enter Connection String
wpwd = "wallet password"  # Enter Wallet Password
dsn = 'paulparkdb_high' # Enter Connection String (from tnsnames.ora)
wpwd = "YourWalletPassword123#"  # Enter Wallet Password

# Use absolute path to wallet directory
wallet_path = os.path.expanduser('~/wallet')

connection = oracledb.connect(
    config_dir=wallet_path,
    user=un, 
    password=pw, 
    dsn=dsn,
    wallet_location=wallet_path,
    wallet_password=wpwd
)

print("Successfully connected to Oracle Database 23ai")

## Step 3: Load PDF Document

Load the PDF document and display basic information about it.

In [ ]:
from PyPDF2 import PdfReader

pdf = PdfReader('oracle-ai-database-26ai-new-features-guide.pdf')
print(f"The number of pages in this document is {len(pdf.pages)}")
print("\n--- First Page Preview ---")
print(pdf.pages[0].extract_text())

## Step 4: Transform PDF to Text

Extract text from all pages of the PDF document.

In [ ]:
if pdf is not None:
    print("Transforming the PDF document to text...")
    text = ""
    for page in pdf.pages:
        text += page.extract_text()
    print(f"Successfully transformed {len(pdf.pages)} pages to text")
    print(f"Total text length: {len(text)} characters")

## Step 5: Split Text into Chunks

Chunk size: 800 characters with 100 character overlap.

**Note:** Chunk sizes vary depending on document type.

In [ ]:
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=800,
    chunk_overlap=100,
    length_function=len
)

chunks = text_splitter.split_text(text)
print(f"Created {len(chunks)} chunks")
print("\n--- First Chunk Preview ---")
print(chunks[0])

## Step 6: Create Documents with Metadata

Wrap each chunk with metadata (id and link) for storage in the vector database.

In [ ]:
docs = [
    chunks_to_docs_wrapper({
        'id': f'{page_num}', 
        'link': f'Page {page_num}', 
        'text': text
    }) 
    for page_num, text in enumerate(chunks)
]

print(f"Created {len(docs)} documents with metadata")

## Step 7: Initialize Vertex AI

Configure your Google Cloud project and region for Vertex AI services.

## Step 6a: Authenticate with GCP (First Time Setup)

**Run this ONCE on your GCP VM to set up authentication:**

```bash
# In the VM terminal (not in notebook):
gcloud auth application-default login --no-launch-browser
```

Follow the URL, copy the authorization code, and paste it back.

**OR** if you have a service account key file:
```bash
export GOOGLE_APPLICATION_CREDENTIALS="/path/to/your/service-account-key.json"
```

⚠️ **After authentication, restart the kernel and re-run from the top.**

In [ ]:
import vertexai

PROJECT_ID = "adb-pm-prod"  # Update with your GCP Project ID
REGION = "us-central1"             # Update with your region (us-central1 has most models)

vertexai.init(project=PROJECT_ID, location=REGION)
print(f"Initialized Vertex AI for project: {PROJECT_ID} in region: {REGION}")

## Step 8: Embed and Store Vectors in Oracle 23ai

Using **VertexAIEmbeddings** model and **DOT_PRODUCT** distance strategy for similarity search.

In [ ]:
from langchain_community.vectorstores.oraclevs import OracleVS
from langchain_community.vectorstores.utils import DistanceStrategy

embeddings = VertexAIEmbeddings(model_name="text-embedding-004")

s1time = time.time()
print(f"Vectorizing and inserting {len(docs)} chunks into Oracle Database 23ai...")
print("Processing in smaller batches to avoid token limits...")

# Process in batches of 50 (to stay under 20k token limit)
batch_size = 50
knowledge_base = None

for i in range(0, len(docs), batch_size):
    batch = docs[i:i + batch_size]
    batch_num = (i // batch_size) + 1
    total_batches = (len(docs) + batch_size - 1) // batch_size
    print(f"  Processing batch {batch_num}/{total_batches} ({len(batch)} chunks)...")
    
    if knowledge_base is None:
        # Create the vector store with first batch
        knowledge_base = OracleVS.from_documents(
            batch,
            embeddings,
            client=connection,
            table_name="RAG_TAB",
            distance_strategy=DistanceStrategy.DOT_PRODUCT
        )
    else:
        # Add remaining batches to existing store
        knowledge_base.add_documents(batch)

s2time = time.time()
print(f"✓ Vectorizing and inserting chunks duration: {round(s2time - s1time, 1)} sec.")
print(f"✓ Successfully stored {len(docs)} chunks in Oracle Database 23ai")

## Step 9: Verify Data in Oracle Database

Query the RAG_TAB table to confirm vectors were inserted successfully.

In [ ]:
table_name = "RAG_TAB"

with connection.cursor() as cursor:
    query = f"SELECT * FROM {table_name}"
    cursor.execute(query)
    rows = cursor.fetchall()
    
    print(f"Total rows in {table_name}: {len(rows)}")
    print("\n--- Sample Rows (first 3) ---")
    for row in rows[:3]:
        print(row)

## Step 10: Define User Question

Ask a question about the document content.

In [ ]:
user_question = 'Tell me more about JSON Relational Duality'
print(f"The prompt to the LLM will be: {user_question}")

## Step 11: Perform Similarity Search

Test the vector similarity search to find relevant chunks.

In [ ]:
if user_question:
    s3time = time.time()
    result_chunks = knowledge_base.similarity_search(user_question, k=5)
    s4time = time.time()
    
    print(f"✓ Search duration: {round(s4time - s3time, 1)} sec.")
    print(f"\nFound {len(result_chunks)} relevant chunks:")
    for i, chunk in enumerate(result_chunks, 1):
        print(f"\nChunk {i}: {chunk.page_content[:200]}...")

## Step 12: Configure Gemini LLM

Set up Vertex AI's **Gemini 2.0 Flash** model for generating responses.

In [ ]:
llm = ChatVertexAI(
    model_name="gemini-2.5-flash",
    max_output_tokens=8192,
    temperature=0.7,
    top_p=0.95,
    top_k=40,
    verbose=True
)

print("✓ Configured Gemini 2.5 Flash model")

## Step 13: Build RAG Prompt Template

Create the prompt template and retriever for the RAG pipeline.

In [ ]:
template = """Answer the question based only on the following context:
            {context} Question: {question} """
prompt = PromptTemplate.from_template(template)
retriever = knowledge_base.as_retriever(search_kwargs={"k": 10})
print("The template is:", template)
print(retriever)

## Step 14: Execute RAG Chain

Invoke the complete RAG pipeline to generate the final response.

The chain:
1. Retrieves relevant context from Oracle Vector DB
2. Constructs prompt with question + context
3. Sends to Gemini LLM for response generation

In [ ]:
s5time = time.time()
print("Sending prompt and RAG context to Gemini LLM...")
print(f"Question: {user_question}\n")

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

response = chain.invoke(user_question)

s6time = time.time()

print("=" * 80)
print("RESPONSE:")
print("=" * 80)
print(response)
print("=" * 80)
print(f"\n✓ LLM response duration: {round(s6time - s5time, 1)} sec.")

## 🎉 Congratulations!

You've successfully built a complete RAG application with:
- **Oracle Database 23ai** for vector storage
- **Vertex AI Embeddings** for vectorization  
- **Gemini 2.0 Flash** for response generation
- **LangChain 1.x** for orchestration

In [ ]:
print("=" * 80)
print("🎉 RAG Application Complete!")
print("=" * 80)
print("\n✓ PDF loaded and processed")
print("✓ Text chunked and vectorized") 
print("✓ Vectors stored in Oracle Database 23ai")
print("✓ Similarity search working")
print("✓ Gemini LLM integration successful")
print("\nYou've completed the RAG application lab!")
print("=" * 80)